In [ ]:
from io import BytesIO
from zipfile import ZipFile
import datetime
from datetime import date, timedelta
import os
import requests
import pandas as pd
import numpy as np
import time
import socket

import os
import requests
import pandas as pd
import numpy as np

from pystac_client import Client
from shapely.geometry import shape, Point

import geopandas as gpd
import matplotlib.pyplot as plt


from datetime import timedelta

from planetary_computer import sign
from stackstac import stack
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import datetime

import hvplot.pandas
import panel as pn
pn.extension('bokeh')

import random

from stackstac import stack
from planetary_computer import sign
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from odc.stac import load


In [ ]:
target_date = pd.to_datetime("2023-05-01").date()

# get ais data

In [ ]:
fname = f"AIS_2023_{target_date.month:02d}_{target_date.day:02d}.zip"
url = f"https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2023/{fname}"

r = requests.get(url, timeout=60)
with ZipFile(BytesIO(r.content)) as z:
    csv_name = z.namelist()[0]
    with z.open(csv_name) as f:
        ais = pd.read_csv(f)
        success = True


# Clean
ais = ais[ais.TransceiverClass == 'A']                      #large cargo ships, not personal vessels
ais = ais[(ais.SOG > 1) & (ais.SOG < 80)]                   #drops ships in harbor, bad data
ais = ais[(ais.Length > 30) & (ais.Length < 400)]           #drops small ships, bad data
ais = ais.replace({'Heading': {511: np.nan}})               #heading 511 is nonrespondor, set to nan

# Sort
ais = ais.sort_values(by=['MMSI', 'BaseDateTime']).reset_index(drop=True)

# Drop weak tracks
mmsi_counts = ais.MMSI.value_counts()
active = mmsi_counts[mmsi_counts >= 5].index
ais = ais[ais["MMSI"].isin(active)].reset_index(drop=True)

#standardize date dtype
ais["BaseDateTime"] = pd.to_datetime(ais["BaseDateTime"], utc=True)

In [ ]:
ais.sample()

# cut down to intersections

In [ ]:
api = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [ ]:
results = api.search(
    collections=["sentinel-2-l2a"],

    bbox=[-160, 4, -50, 50],
    datetime=str(target_date),
    query={"eo:cloud_cover": {"lt": 30}}
)

In [ ]:
results = api.search(
    collections=["sentinel-1-grd"],

    bbox=[-160, 4, -50, 50],
    datetime='2023-05-01',#str(target_date),
)

In [ ]:
sentinel_passes = []
for item in results.get_all_items():
    geom = shape(item.geometry)
    dt = item.datetime
    sentinel_passes.append((geom, dt))

print(len(sentinel_passes))

In [ ]:
geoms = [poly for poly, _ in sentinel_passes]
times = [dt for _, dt in sentinel_passes]

gdf = gpd.GeoDataFrame({'datetime': times}, geometry=geoms, crs='EPSG:4326')

In [ ]:
gdf.hvplot(
    geo=True,
    tiles='CartoLight',
    line_color='blue',
    line_width=0.5,
    frame_width=900,
    frame_height=600
)

In [ ]:
# Prep sentinel GeoDataFrame
poly_list = []
time_list = []

filtered_passes = [
    (poly, dt) for poly, dt in sentinel_passes
    if dt.date() == target_date
]

poly_list = [poly for poly, _ in filtered_passes]
time_list = [dt for _, dt in filtered_passes]

sentinel_gdf = gpd.GeoDataFrame({'datetime': time_list}, geometry=poly_list, crs='EPSG:4326')

In [ ]:
ais_gdf = gpd.GeoDataFrame(
    ais,
    geometry=gpd.points_from_xy(ais["LON"], ais["LAT"]),
    crs='EPSG:4326'
)

# We'll accumulate matching AIS rows into this list
matching_rows = []

# Define time window in seconds
TIME_WINDOW = timedelta(minutes=10)

# Iterate over sentinel polygons
for _, s2_row in sentinel_gdf.iterrows():
    poly = s2_row.geometry
    s2_time = s2_row.datetime

    # Filter AIS to time window first (fast)
    time_mask = (ais_gdf["BaseDateTime"] >= s2_time - TIME_WINDOW) & \
                (ais_gdf["BaseDateTime"] <= s2_time + TIME_WINDOW)
    candidate_ais = ais_gdf[time_mask]

    # Spatial filter: points inside this polygon
    inside_mask = candidate_ais.geometry.intersects(poly)
    intersecting = candidate_ais[inside_mask]

    if not intersecting.empty:
        matching_rows.append(intersecting)

# Concatenate results into one DataFrame
ais_intersecting = pd.concat(matching_rows, ignore_index=True)

In [ ]:
len(ais)

In [ ]:
len(ais_intersecting)

In [ ]:
# Convert gulf_df into GeoDataFrame
ais_gdf = gpd.GeoDataFrame(
    ais_intersecting,
    geometry=gpd.points_from_xy(ais_intersecting["LON"], ais_intersecting["LAT"]),
    crs="EPSG:4326"
)

# Optional: filter AIS to a single day to avoid clutter
ais_gdf["BaseDateTime"] = pd.to_datetime(ais_gdf["BaseDateTime"])
ais_day = ais_gdf[
    (ais_gdf["BaseDateTime"].dt.date == target_date)
]

# Plot S2 footprints + AIS pings
fig, ax = plt.subplots(figsize=(10, 6))
gdf.plot(ax=ax, edgecolor='blue', facecolor='none', linewidth=0.5)
ais_day.plot(ax=ax, color='red', markersize=2, alpha=0.5)

ax.set_title('Sentinel-2 Overpasses + AIS Ship Positions (2023-01-01)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.grid(True)
plt.show()

# get sat image

In [ ]:
api = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [ ]:
row = ais_intersecting.iloc[random.randint(0,len(ais_intersecting))]
print(f"Selected MMSI: {row['MMSI']} @ {row['BaseDateTime']}")

ais_point = Point(row["LON"], row["LAT"])
time_buffer = timedelta(minutes=30)

In [ ]:
# Load image with a slightly bigger bbox (0.2° instead of 0.1°)
search = api.search(
    collections=["sentinel-1-grd"],
    bbox=[row["LON"] - 0.2, row["LAT"] - 0.2, row["LON"] + 0.2, row["LAT"] + 0.2],
    datetime=f"{(row['BaseDateTime'] - time_buffer).isoformat()}/{(row['BaseDateTime'] + time_buffer).isoformat()}",
    limit=5,
    query={
        "sar:instrument_mode": {"eq": "IW"},
    }
)

items = list(search.get_items())

tile_geom = shape(items[0].geometry)
ais_point = Point(row["LON"], row["LAT"])

print("Tile bounds:", tile_geom.bounds)
print("AIS point inside tile?", tile_geom.contains(ais_point))

In [ ]:
item = sign(items[0])

arr = load(
    [item],
    bands=['vv', 'vh'],
    crs="EPSG:4326",  # Important: forces lat/lon coordinates
    resolution=0.0001,  # About 10m at equator
    bbox=(row["LON"] - 0.2, row["LAT"] - 0.2, row["LON"] + 0.2, row["LAT"] + 0.2),
    groupby="solar_day"
)

In [ ]:
print(arr)

In [ ]:
mode = 'vv'

In [ ]:
lat = row["LAT"]
lon = row["LON"]
buffer = 0.1

# Make sure latitude is decreasing (north to south)
lat_slice = slice(lat + buffer, lat - buffer)
lon_slice = slice(lon - buffer, lon + buffer)

crop = arr[mode].sel(
    latitude=lat_slice,
    longitude=lon_slice
).isel(time=0)


In [ ]:
arr[mode].shape

In [ ]:
crop.shape

In [ ]:
import xarray as xr
import hvplot.xarray

img = crop


vmin, vmax = np.percentile(img.values[~np.isnan(img.values)], (2, 98))
img_clipped = img.clip(min=vmin, max=vmax)

plot = img_clipped.hvplot.image(
    x='longitude',
    y='latitude',
    cmap='gray',
    frame_width=600,
    frame_height=600,
    title=f"Sentinel-1 {mode} @ {lat:.2f}, {lon:.2f}",
    invert=True
)

plot 

In [ ]:
row[['MMSI', 'VesselName', 'SOG', 'Heading', 'Length']]